In [1]:
import pandas as pd
import numpy as np
import pymysql

from datetime import datetime
from importlib import reload
from WindPy import w
import cvxpy as cp

In [3]:
def getHS300Data(conn, start_dt, end_dt):
    '''
       Function description : 提取沪深300成分股信息
       param : conn - 数据连接
       return :
       Usage:

    '''
    query = f"""
    SELECT s_con_windcode, trade_dt, i_weight
    FROM aindexhs300weight
    WHERE trade_dt >= '{str(start_dt)}' AND trade_dt <= '{str(end_dt)}'
    ORDER BY s_con_windcode ASC
    """
    data = pd.read_sql_query(query, conn)
    return data

conn = pymysql.connect(host="192.168.64.57", user="infoasadep01", password="tfyfInfo@1522", database="wind", port=3306, charset="utf8")
data = getHS300Data(conn, 20230105, 20230105)

C:\Users\17509\AppData\Local\Temp\ipykernel_11684\2701659959.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, conn)


In [4]:
id_list = data.iloc[:, 0].tolist()
stock_id_list = ', '.join(id_list)
# 初始化Wind API
w.start()

#获取风格因子数据
df = w.wss(stock_id_list, "mv_ref,pq_avgturn2,roe_ttm2,qfa_yoyprofit,pct_chg_per,industry_citic,val_lnmv,eqy_belongto_parcomsh","unit=1;tradeDate=20240105;startDate=20231204;endDate=20240105;rptDate=20230930;industryType=1;rptType=1")
df = pd.DataFrame(df.Data, columns=df.Codes, index=df.Fields).T

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [5]:
import os

out_folder = 'wind_data_prj/'
os.makedirs(out_folder,exist_ok=True)
df.to_csv(out_folder + '风格因子.csv')

In [ ]:

import pickle

def write_pickle(data,filename):
    with open(filename,'wb') as f:
        pickle.dump(data,f)
        return

def read_pickle(filename):
    with open(filename,'rb') as f:
        return pickle.load(f)

write_pickle(df,out_folder + '风格因子.pkl')

In [15]:
column_mapping = {
    'MV_REF': '总市值',
    'PQ_AVGTURN2': '换手率',
    'ROE_TTM2': 'ROETTM',
    'QFA_YOYPROFIT': '净利润增速',
    'PCT_CHG_PER': '反转',
    'INDUSTRY_CITIC': '所属行业',
    'VAL_LNMV': '市值对数',
    'EQY_BELONGTO_PARCOMSH': '净资产'
}


df.rename(columns=column_mapping, inplace=True)

df['BP'] = df['净资产']/df['总市值']
style_factors = df[['换手率', 'ROETTM', '净利润增速', '反转', '市值对数', 'BP']]

print(style_factors)

                换手率   ROETTM    净利润增速        反转     市值对数        BP
000001.SZ  0.460171  10.4091  -2.2167  -4.03727  25.9156   2.58965
000002.SZ  0.669954   7.5873 -36.0073  -12.3684   25.504   2.12193
000063.SZ   2.05605  13.7335  10.3084  -6.08263  25.4891  0.564361
000069.SZ  0.600796 -22.0902 -2093.79  -14.7399  23.9094   2.69013
000100.SZ   1.23421   3.0423  12497.8   1.19617  25.0982  0.658792
...             ...      ...      ...       ...      ...       ...
688363.SH  0.989335   11.664 -57.2903   -9.5914  24.1369  0.228029
688396.SH  0.297412   7.7214 -60.5066  -9.31362  24.7608  0.369238
688561.SH  0.718434  -0.4546 -58.7363  -18.1272   23.951  0.355942
688599.SH  0.663595   20.256  77.7948 -0.597684  24.7809  0.542413
688981.SH  0.866518    4.488 -70.6928   -6.5928  26.7196  0.355806

[300 rows x 6 columns]


In [ ]:
# 使用get_dummies将"所属行业"列转化为独热编码
industry_dummies = pd.get_dummies(df['所属行业'])

# 行业因子矩阵
industry_factors = pd.DataFrame(industry_dummies)

#因子数据
factor_scores = w.wss(stock_id_list,"qfa_roe","rptDate=20230930")
HS300_factor = w.wss("000300.SH", "qfa_roe","rptDate=20230930")

In [9]:
#计算基准指数风格因子矩阵
merged_data = style_factors.merge(data[['s_con_windcode', 'i_weight']], left_index=True, right_on='s_con_windcode')
merged_data = merged_data.drop(columns='s_con_windcode')

merged_data = merged_data.set_index(style_factors.index)
selected_columns = merged_data.iloc[:, :6]
weighted_columns = selected_columns.mul(merged_data['i_weight']/100, axis=0)
weighted_sum = weighted_columns.sum()

HS300_style = pd.DataFrame(weighted_sum.values.reshape(1,-1), index=["HS300"], columns=selected_columns.columns)

In [10]:
column_mapping = {
    'MV_REF': '总市值',
    'PQ_AVGTURN2': '换手率',
    'ROE_TTM2': 'ROETTM',
    'QFA_YOYPROFIT': '净利润增速',
    'PCT_CHG_PER': '反转',
    'INDUSTRY_CITIC': '所属行业',
    'VAL_LNMV': '市值对数',
    'EQY_BELONGTO_PARCOMSH': '净资产'
}


df.rename(columns=column_mapping, inplace=True)

df['BP'] = df['净资产']/df['总市值']
style_factors = df[['换手率', 'ROETTM', '净利润增速', '反转', '市值对数', 'BP']]

# 风格因子
print(style_factors)

                换手率   ROETTM       净利润增速         反转     市值对数        BP
000001.SZ  0.460171  10.4091     -2.2167  -4.037267  25.9156  2.589653
000002.SZ  0.669954   7.5873    -36.0073 -12.368421   25.504  2.121928
000063.SZ  2.056053  13.7335     10.3084  -6.082632  25.4891  0.564361
000069.SZ  0.600796 -22.0902  -2093.7937 -14.739884  23.9094  2.690131
000100.SZ  1.234211   3.0423  12497.7574   1.196172  25.0982  0.658792
...             ...      ...         ...        ...      ...       ...
688363.SH  0.989335   11.664    -57.2903  -9.591398  24.1369  0.228029
688396.SH  0.297412   7.7214    -60.5066  -9.313622  24.7608  0.369238
688561.SH  0.718434  -0.4546    -58.7363 -18.127224   23.951  0.355942
688599.SH  0.663595   20.256     77.7948  -0.597684  24.7809  0.542413
688981.SH  0.866518    4.488    -70.6928  -6.592798  26.7196  0.355806

[300 rows x 6 columns]


In [11]:
# 使用get_dummies将"所属行业"列转化为独热编码
industry_dummies = pd.get_dummies(df['所属行业'])

# 行业因子矩阵
industry_factors = pd.DataFrame(industry_dummies)

# 因子数据
factor_scores = w.wss(stock_id_list,"qfa_roe","rptDate=20230930")
HS300_factor = w.wss("000300.SH", "qfa_roe","rptDate=20230930")

In [12]:
# 计算基准指数风格因子矩阵
merged_data = style_factors.merge(data[['s_con_windcode', 'i_weight']], left_index=True, right_on='s_con_windcode')
merged_data = merged_data.drop(columns='s_con_windcode')

merged_data = merged_data.set_index(style_factors.index)
selected_columns = merged_data.iloc[:, :6]
weighted_columns = selected_columns.mul(merged_data['i_weight']/100, axis=0)
weighted_sum = weighted_columns.sum()

HS300_style = pd.DataFrame(weighted_sum.values.reshape(1,-1), index=["HS300"], columns=selected_columns.columns)

In [13]:
num_stocks = df.shape[0]
w_b = np.array(data['i_weight']/100)  
factor_values = np.array(factor_scores.Data[0])  
X = np.array(style_factors)  
H = np.array(industry_factors)  

In [14]:
# 初始化股票权重为优化变量
w = cp.Variable(num_stocks)

# 目标函数：最大化因子暴露
objective = cp.Maximize(factor_values.T @ w)

In [9]:
# 设置约束条件的参数


# HS300_style_sl = HS300_style.copy()
# HS300_style_sl.iloc[:, :4] += 9999999999
# HS300_style_sl.iloc[:, 5:] += 9999999999
# s_l = np.array(HS300_style_sl) 
# s_l = s_l.reshape(-1)

# HS300_style_sh = HS300_style.copy()
# HS300_style_sh.iloc[:, :4] += -99999999999
# HS300_style_sh.iloc[:, 5:] += -99999999999
# s_h = np.array(HS300_style_sh) 
# s_h = s_h.reshape(-1)



# 市值对数这列数据保持不变，其他列都设为无穷
HS300_style_sl = np.copy(HS300_style)
HS300_style_sl[:, :4] = np.inf
HS300_style_sl[:, 5:] = np.inf
s_l = HS300_style_sl.reshape(-1)

HS300_style_sh = np.copy(HS300_style)
HS300_style_sh[:, :4] = -np.inf
HS300_style_sh[:, 5:] = -np.inf
s_h = HS300_style_sh.reshape(-1)

h_l = 0.5 * np.ones((27,))  
h_h = 1.5 * np.ones((27,))  

w_l = w_b - 0.01 * np.ones(300)
w_h = w_b + 0.01 * np.ones(300)

# b_l, b_h = 0, 1  # 成分股权重占比的上下限

l = 2  # 个股权重上限

In [10]:
# 定义约束
constraints = [
    s_l.T <= X.T @ (w - w_b),  # 风格暴露约束
    X.T @ (w - w_b) <= s_h.T,

    h_l <= H.T @ (w - w_b),  # 行业暴露约束
    H.T @ (w - w_b) <= h_h,

    w_l <= w - w_b,
    w - w_b <= w_h,

    # b_l <= w,  # 成分股权重占比约束
    # w <= b_h,

    0 <= w,
    # w <= l,

    cp.sum(w) == 1,  # 权重和为1
]

In [11]:
# 求解优化问题
prob = cp.Problem(objective, constraints)

prob.solve(solver=cp.ECOS, verbose=True)

                                     CVXPY                                     
                                    v1.1.24                                    
(CVXPY) Feb 02 10:49:45 AM: Your problem has 300 variables, 8 constraints, and 0 parameters.
(CVXPY) Feb 02 10:49:45 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 02 10:49:45 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Feb 02 10:49:45 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 02 10:49:45 AM: Compiling problem (target solver=ECOS).
(CVXPY) Feb 02 10:49:45 AM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> Con

c:\Users\liyou\AppData\Local\Programs\Python\Python36-32\lib\site-packages\cvxpy\problems\problem.py:1303: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


-3.944429438117649e+82

In [12]:
print(w.value)

[-4.24035886e+79 -4.11873337e+79 -4.30237770e+79 -8.91087012e+78
 -1.86641918e+78 -3.78491894e+79 -4.31283203e+79 -1.10693369e+79
 -4.13238624e+79 -3.13967204e+79 -4.11139383e+79 -4.39114225e+79
 -3.84219412e+79 -4.08187969e+79 -4.01368012e+79 -3.74671606e+79
 -4.23579703e+79 -4.41320188e+79 -4.37698029e+79 -3.83001534e+79
 -3.57991641e+79 -4.51953442e+79 -4.19786298e+79 -4.31541696e+79
 -4.45441093e+79 -3.79059889e+79 -3.74485298e+79 -4.09673769e+79
 -3.52763698e+79 -3.51945175e+79 -4.25909565e+79 -4.41713221e+79
 -4.39318003e+79 -4.45135838e+79 -4.25376179e+79 -4.33721113e+79
 -4.42394471e+79 -3.82902177e+79 -4.30088010e+79 -3.73808236e+79
 -4.14120550e+79 -4.42538760e+79 -4.32508780e+79 -7.01231730e+78
 -4.48194199e+79 -4.22037591e+79 -4.41114416e+79 -4.28880636e+79
 -4.36369212e+79 -3.61983944e+79 -3.79637291e+79 -3.77573233e+79
 -3.21726505e+79 -3.82919614e+79 -4.42670590e+79 -4.08660427e+79
 -4.26348621e+79 -4.37878928e+79 -4.23466893e+79 -4.29421211e+79
 -3.84352346e+79 -4.16286

In [18]:
result_df = pd.DataFrame({'Result': factor_values * w.value}, index=factor_scores.Codes)
#降序排列
result_df1 = result_df.sort_values(by="Result", ascending=False)
# 选择前20%的数据并计算平均值
percentile = 0.2
num_rows_to_select = int(num_stocks * percentile)
average_value = result_df.head(num_rows_to_select)["Result"].mean()

# 计算超额收益
excess_returns = average_value - HS300_factor.Data
print("{:.2f}%".format(float(excess_returns[0]) * 100))


-12459255703170529812515405425618757474132041146059394279711790483735661921429356544.00%
